<a href="https://colab.research.google.com/github/ZeyadAboeleneen/Deep_Learning/blob/main/Transformar_Ass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 145.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 142.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 M

In [ ]:
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import os
import numpy as np

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
dataset = load_dataset("squad_v2")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [ ]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(500))
small_val_dataset = dataset["validation"].shuffle(seed=42).select(range(100))

In [ ]:
def preprocess_data(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        if len(answer["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_dataset = small_train_dataset.map(preprocess_data, batched=True, remove_columns=small_train_dataset.column_names)
val_dataset = small_val_dataset.map(preprocess_data, batched=True, remove_columns=small_val_dataset.column_names)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
training_args = TrainingArguments(
    output_dir="./trained_qa_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    learning_rate=3e-5,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,4.898300,3.167227


TrainOutput(global_step=63, training_loss=4.696515764508929, metrics={'train_runtime': 1735.9838, 'train_samples_per_second': 0.289, 'train_steps_per_second': 0.036, 'total_flos': 65587856216064.0, 'train_loss': 4.696515764508929, 'epoch': 1.0})

In [ ]:
def debug_qa(question, context):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True)
    print("Tokenized input:", tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

    with torch.no_grad():
        outputs = model(**inputs)

    start_probs = torch.softmax(outputs.start_logits, dim=1)[0]
    end_probs = torch.softmax(outputs.end_logits, dim=1)[0]
    top_starts = torch.topk(start_probs, 5)
    top_ends = torch.topk(end_probs, 5)

    print("\nTop start positions:")
    for i, (prob, idx) in enumerate(zip(top_starts.values, top_starts.indices)):
        token = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][idx])
        print(f"{i+1}. Prob: {prob:.2f}, Token: '{token}' (position {idx})")

    print("\nTop end positions:")
    for i, (prob, idx) in enumerate(zip(top_ends.values, top_ends.indices)):
        token = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][idx])
        print(f"{i+1}. Prob: {prob:.2f}, Token: '{token}' (position {idx})")

    best_start = torch.argmax(start_probs)
    best_end = torch.argmax(end_probs)

    answer_tokens = inputs["input_ids"][0][best_start:best_end+1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    print("\nRaw model output:", answer)
    return answer

In [ ]:
context_text = """Diabetes is a chronic disease that occurs when the pancreas is no longer able to make insulin,
or when the body cannot make good use of the insulin it produces. Common symptoms include increased thirst,
frequent urination, and extreme fatigue."""

question_text = "what are symptoms of diabetes"
answer = ask_question(question_text, context_text)
print(f"Question: {question_text}")
print(f"Answer: {answer}")

Question: what are symptoms of diabetes
Answer: increased thirst,
frequent urination, and extreme fatigue
